In [19]:
## Import necessary modules
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import AutoDateFormatter, AutoDateLocator, WeekdayLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
import numpy as np
import datetime
%matplotlib gtk

In [20]:
## Import data frome pickle generated from hotline_daily.py
data = pd.read_pickle('pickle_hotline_0904.pkl')

In [21]:
## Have a look at the data structure
data.head(3)

,month,year,0,1,2,3,4,5,6,7,...,21,22,23,24,summa,xlday,day,date,week,weekday
tix,11,2016,00:00-00:30,00:30-01:30,01:30-02:30,02:30-03:30,03:30-04:30,04:30-05:30,05:30-06:30,06:30-07:30,...,20:30-21:30,21:30-22:30,22:30-23:30,23:30-00:00,NaN,42688.0,14,2016-11-14,46,Mon
angekommen,11,2016,0,0,0,0,0,0,1,5,...,0,1,0,0,321,42688.0,14,2016-11-14,46,Mon
verbunden,11,2016,0,0,0,0,0,0,0,4,...,0,0,0,0,221,42688.0,14,2016-11-14,46,Mon


In [22]:
## separate the sums of calls, verbundene and lost into different dataframes; this will avoid tuple indexing
calls_d=data[['date','summa','weekday','day']].loc['angekommen'].set_index("date")
verbu_d=data[['date','summa','weekday','day']].loc['verbunden'].set_index("date")
verlo_d=data[['date','summa','weekday','day']].loc['verloren'].set_index("date")

## have a look at one of the new dataframes
calls_d.head(2)

,summa,weekday,day
date,,,
2016-11-14,321,Mon,14
2016-11-15,152,Tue,15


In [23]:
## merge dataframes again into one, containing the sums and indexed by date.
## this seems kind of redundant, given the frames above. guess it should be refactored; but works for now

dayf = pd.DataFrame(index=calls_d.index)
dayf[['angekommen','weekday','day']]=calls_d[['summa','weekday','day']]
dayf['verbunden']=verbu_d['summa']
dayf['verloren']=verlo_d['summa']
dayf=dayf[['weekday','day','angekommen','verbunden','verloren']]

## take a glance at the new dataframe
dayf.head(2)

,weekday,day,angekommen,verbunden,verloren
date,,,,,
2016-11-14,Mon,14,321,221,100
2016-11-15,Tue,15,152,132,20


In [25]:
## this will generate the ticks on the x-Axis of the plot
## I don't fully understand this, though. more reading is due
xtick_locator = WeekdayLocator(byweekday=MO)
xtick_formatter = DateFormatter('%W/%y')

#print(xtick_locator)
#print(xtick_formatter)



In [28]:
## Create the plot from the summary dataframe
## X-Axis should be dates, Y-Axis should be number of calls

fig = plt.figure(figsize=(18,6))
fig.add_subplot(111)
plt.ylabel('Calls')
plt.xlabel('Kalenderwoche')
plt.margins(0.01)
plt.title('Anrufaufkommen Carexpert seit 14.11.2016')

# "Initialize" Axes to configure them later
ax = plt.axes()
ax.set_axisbelow(True)

# Y-Axis: major ticks every 50, minor ticks every 10                                    
minor_ticks = np.arange(0, dayf.angekommen.max(), 10)
major_ticks = np.arange(0, dayf.angekommen.max(), 50)
ax.yaxis.grid(True)
ax.set_yticks(minor_ticks, minor=True)
ax.yaxis.grid(True, which='minor', color='#E8E6BF', linestyle='--', alpha=0.5, zorder=0)
ax.yaxis.grid(True, which='major', color='#6DC3EC', linestyle='-', zorder=3)

# X-Axis: locator set the ticks. No understanding of this, just copy-and-paste...
ax.xaxis.set_major_locator(xtick_locator)
ax.xaxis.set_major_formatter(xtick_formatter)

# Bars for each sum and day
plt.bar(dayf.index,dayf.angekommen,width=0.9, alpha=1, color='#8F8F8F', label='angekommen', align='center')
plt.bar(dayf.index,dayf.verbunden,width=0.2, alpha=1, color='#00B9FF', label='verbunden', align='center')
plt.bar(dayf.index,dayf.verloren,width=0.2, alpha=1, color='#FF0056', label='verloren', align='center')

# Markers for Mondays and Months
plt.plot(dayf[dayf.weekday=='Mon'].index, dayf[dayf.weekday=='Mon'].day, c='#00FFEB', marker='d', markersize='4', linestyle='none', label='Montage')
plt.plot(dayf[dayf.day==1].index, dayf[dayf.day==1].day, c='#FFF800', marker='^', markersize='4', linestyle='none', label='Monatsbeginn')

# Legend and Layout formatting after the plotting has taken place
plt.legend()
plt.tight_layout()
